# Stan models for NFL games

Using Stan to infer team qualities based on football games up to week 13 in the 2016 football season.

The probability that the home team (h) beats the away team (a) is modeled by
$$ P(\text{h beats a}) = \text{logistic}(\alpha + \beta_h - \beta_a) $$

where $\alpha$ is the intercept that models home-field advantage, and $\beta_i$ is the quality of team $i$.


[Data](http://www.pro-football-reference.com/years/2016/games.htm)

In [7]:
import re

import pandas as pd
import numpy as np

import toyplot as tp
import matplotlib.pyplot as plt
%matplotlib inline

import pystan as ps

In [8]:
raw_df = pd.read_csv("data/2016-week13.csv").drop("NA", 1)
raw_df.head()

,Week,Day,Date,Time,Winner,Location,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,Thu,September 8,8:40PM,Denver Broncos,NaN,Carolina Panthers,21.0,20.0,307.0,3.0,333.0,1.0
1,1,Sun,September 11,1:04PM,Baltimore Ravens,NaN,Buffalo Bills,13.0,7.0,308.0,1.0,160.0,0.0
2,1,Sun,September 11,1:04PM,Green Bay Packers,@,Jacksonville Jaguars,27.0,23.0,294.0,0.0,348.0,1.0
3,1,Sun,September 11,1:05PM,Houston Texans,NaN,Chicago Bears,23.0,14.0,344.0,1.0,258.0,1.0
4,1,Sun,September 11,1:05PM,Minnesota Vikings,@,Tennessee Titans,25.0,16.0,301.0,0.0,316.0,3.0


In [9]:
team_to_id = { team: idx for idx, team in enumerate(raw_df["Loser"].factorize()[1]) }
id_to_team = { idx: team for idx, team in enumerate(raw_df["Loser"].factorize()[1]) }

def team_id(name):
    """ Function to find the team id corresponding to a team name"""
    try:
        return [idx for team, idx in team_to_id.items() if name.lower() in team.lower()][0]
    except:
        return None
    
def find_games(data, team):
    return data.loc[(traingames["tid"]==team_id(team)) | (traingames["oid"]==team_id(team))]

In [10]:
raw_df["win_id"] = raw_df.apply(lambda x: team_to_id[x.Winner], 1)
raw_df["los_id"] = raw_df.apply(lambda x: team_to_id[x.Loser], 1)
raw_df["at_home"] = raw_df.apply(lambda x: 0 if x.Location == "@" else 1, 1)

In [16]:
df = raw_df[raw_df["Week"] <= 14]
df.tail()

,Week,Day,Date,Time,Winner,Location,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,win_id,los_id,at_home
203,14,Sun,December 11,4:25PM,New Orleans Saints,@,Tampa Bay Buccaneers,NaN,NaN,NaN,NaN,NaN,NaN,9,20,0
204,14,Sun,December 11,4:25PM,Seattle Seahawks,@,Green Bay Packers,NaN,NaN,NaN,NaN,NaN,NaN,21,23,0
205,14,Sun,December 11,4:25PM,Atlanta Falcons,@,Los Angeles Rams,NaN,NaN,NaN,NaN,NaN,NaN,8,15,0
206,14,Sun,December 11,8:30PM,Dallas Cowboys,@,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,11,25,0
207,14,Mon,December 12,8:30PM,Baltimore Ravens,@,New England Patriots,NaN,NaN,NaN,NaN,NaN,NaN,27,28,0


In [17]:
dict_df = df.to_dict(orient="index")

In [18]:
def extract_team_data(x):
    game_w = {"tid": x["win_id"], 
              "oid": x["los_id"],
              "win": 1,
              "at_home": x["at_home"],
              "week": x["Week"],
              "yds": x["YdsW"],
              "opp_yds" : x["YdsL"],
              "tow": x["TOW"],
              "tol": x["TOL"],
              "ptsf": x["PtsW"],
              "ptsa": x["PtsL"],
              "team": x["Winner"],
              "opp": x["Loser"]}
    game_l = {"tid": x["los_id"], 
              "oid": x["win_id"],
              "win": 0,
              "at_home": 1-x["at_home"],
              "week": x["Week"],
              "yds": x["YdsL"],
              "opp_yds" : x["YdsW"],
              "tow": x["TOL"],
              "tol": x["TOW"],
              "ptsf": x["PtsL"],
              "ptsa": x["PtsW"],
              "team": x["Loser"],
              "opp": x["Winner"]
            }
    if game_w["at_home"] == 1:
        return  game_w
    return game_l

In [19]:
allgames = pd.DataFrame([extract_team_data(x) for x in dict_df.values()])
allgames.head()

,at_home,oid,opp,opp_yds,ptsa,ptsf,team,tid,tol,tow,week,win,yds
0,1,0,Carolina Panthers,333.0,20.0,21.0,Denver Broncos,30,1.0,3.0,1,1,307.0
1,1,1,Buffalo Bills,160.0,7.0,13.0,Baltimore Ravens,27,0.0,1.0,1,1,308.0
2,1,23,Green Bay Packers,294.0,27.0,23.0,Jacksonville Jaguars,2,0.0,1.0,1,0,348.0
3,1,3,Chicago Bears,258.0,14.0,23.0,Houston Texans,24,1.0,1.0,1,1,344.0
4,1,31,Minnesota Vikings,301.0,25.0,16.0,Tennessee Titans,4,0.0,3.0,1,0,316.0


In [20]:
traingames = allgames[allgames["week"] < 14]
testgames = allgames[(allgames["week"] >= 14)]
testgames.head()

,at_home,oid,opp,opp_yds,ptsa,ptsf,team,tid,tol,tow,week,win,yds
192,1,22,Oakland Raiders,NaN,NaN,NaN,Kansas City Chiefs,18,NaN,NaN,14,0,NaN
193,1,3,Chicago Bears,NaN,NaN,NaN,Detroit Lions,17,NaN,NaN,14,0,NaN
194,1,30,Denver Broncos,NaN,NaN,NaN,Tennessee Titans,4,NaN,NaN,14,0,NaN
195,1,16,Cincinnati Bengals,NaN,NaN,NaN,Cleveland Browns,5,NaN,NaN,14,0,NaN
196,1,31,Minnesota Vikings,NaN,NaN,NaN,Jacksonville Jaguars,2,NaN,NaN,14,0,NaN


In [21]:
find_games(traingames, "denver")

,at_home,oid,opp,opp_yds,ptsa,ptsf,team,tid,tol,tow,week,win,yds
0,1,0,Carolina Panthers,333.0,20.0,21.0,Denver Broncos,30,1.0,3.0,1,1,307.0
29,1,12,Indianapolis Colts,253.0,20.0,34.0,Denver Broncos,30,2.0,1.0,2,1,400.0
33,1,30,Denver Broncos,355.0,29.0,17.0,Cincinnati Bengals,16,1.0,2.0,3,0,332.0
58,1,30,Denver Broncos,307.0,27.0,7.0,Tampa Bay Buccaneers,20,0.0,3.0,4,0,215.0
71,1,8,Atlanta Falcons,372.0,23.0,16.0,Denver Broncos,30,1.0,1.0,5,0,267.0
77,1,30,Denver Broncos,304.0,13.0,21.0,San Diego Chargers,7,2.0,2.0,6,1,265.0
106,1,24,Houston Texans,271.0,9.0,27.0,Denver Broncos,30,2.0,0.0,7,1,347.0
116,1,7,San Diego Chargers,369.0,19.0,27.0,Denver Broncos,30,3.0,3.0,8,1,324.0
131,1,30,Denver Broncos,299.0,20.0,30.0,Oakland Raiders,22,2.0,0.0,9,1,397.0
141,1,30,Denver Broncos,337.0,25.0,23.0,New Orleans Saints,9,2.0,4.0,10,0,373.0


In [40]:
stan_simple = """
data {
    int<lower=0> nteams; // number of teams
    int<lower=0> N; // number of observations
    int win[N]; // Outcome
    int tid[N]; // Team
    int oid[N]; // Opponent
    
    int Nnew; // new observations
    int tidnew[Nnew];
    int oidnew[Nnew];
}
transformed data {}
parameters {
    real home;
    real team[nteams];
    
    real<lower=0, upper=3> sigma_team;
    real<lower=0, upper=1> sigma_home;
}
transformed parameters {
    vector[N] xb;
    vector[N] pwin;
    for(i in 1:N) {
        xb[i] <- home + team[tid[i]] - team[oid[i]];
        pwin[i] <- inv_logit(xb[i]);
    }
}
model {
    team ~ normal(0, sigma_team);
    home ~ normal(0.1, sigma_home);
    win ~ bernoulli(pwin);
}
generated quantities {
    real xb_n [Nnew];
    real pwin_n [Nnew];
    
    for(i in 1:Nnew) {
        xb_n[i] <- home + team[tidnew[i]] - team[oidnew[i]];
        pwin_n[i] <- inv_logit(xb_n[i]);
    }
}
"""

data_simple = {
    "nteams": 32,
    "N": len(traingames),
    "win": traingames.win,
    "tid": traingames.tid+1,
    "oid": traingames.oid+1,
    "Nnew": len(testgames),
    "tidnew": testgames.tid+1,
    "oidnew": testgames.oid+1,
}

In [41]:
niter_simple = 1000
fit_simple = ps.stan(model_code=stan_simple, data=data_simple, iter=niter_simple, chains=2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_756beea4b3d6e7130190ca99d0ec9a64 NOW.
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)


In [43]:
def team_scores(fit, team, var="team"):
    return sorted(fit.extract()[var][:, team_id(team)])

### Plot of team qualities

Plots team quality and 50% credible interval based on fitted model.

In [44]:
alpha=0.25

canvas = tp.Canvas()
axes = canvas.cartesian(label="Team qualities", xlabel="Score", ylabel="Team", xmin=-2.8)

data = [(idx, team, np.mean(team_scores(fit_simple, team)), team_scores(fit_simple, team)) 
        for idx, team in id_to_team.items()]

data.sort(key=lambda x: x[2])

axes.hlines([x - 0.5 for x in range(1,32)], opacity=0.1)
axes.vlines([0], opacity=0.1)

for idx, (_, _, _, scores) in enumerate(data):
    axes.plot([scores[int(alpha*niter_simple)], scores[int((1-alpha)*niter_simple)]], [idx, idx])

ids = list(range(32))
_, teams, avg_scores, _ = zip(*data)
axes.scatterplot(avg_scores, ids, color="grey")

axes.y.ticks.locator = tp.locator.Explicit(labels=teams)
axes.y.ticks.labels.angle = 270
axes.y.spine.show = False

-3 -2 -1 0 1 2 Score Cleveland Browns San Francisco 49ers Jacksonville Jaguars New York Jets Chicago Bears Carolina Panthers Los Angeles Rams Cincinnati Bengals Arizona Cardinals New Orleans Saints Philadelphia Eagles San Diego Chargers Tennessee Titans Indianapolis Colts Buffalo Bills Minnesota Vikings Houston Texans Green Bay Packers Miami Dolphins Baltimore Ravens Tampa Bay Buccaneers Pittsburgh Steelers Washington Redskins Atlanta Falcons Detroit Lions New York Giants Denver Broncos Seattle Seahawks Kansas City Chiefs New England Patriots Oakland Raiders Dallas Cowboys Team Team qualities 
 
 
 Save as .csv

In [49]:
posterior_preds = fit_simple.extract()["pwin"]
posterior_preds_test = fit_simple.extract()["pwin_n"]
testgames["pwin"] = np.mean(posterior_preds_test, 0)

testgames[["team", "opp", "pwin"]]

,team,opp,pwin
192,Kansas City Chiefs,Oakland Raiders,0.523054
193,Detroit Lions,Chicago Bears,0.792886
194,Tennessee Titans,Denver Broncos,0.442599
195,Cleveland Browns,Cincinnati Bengals,0.321449
196,Jacksonville Jaguars,Minnesota Vikings,0.344441
197,Buffalo Bills,Pittsburgh Steelers,0.493968
198,Carolina Panthers,San Diego Chargers,0.490778
199,Philadelphia Eagles,Washington Redskins,0.449981
200,Miami Dolphins,Arizona Cardinals,0.674589
201,Indianapolis Colts,Houston Texans,0.541453


Model predicts the Chiefs to beat the Raiders with probability 52%, etc.

In [32]:
canvas = tp.Canvas(500, 300)
axes = canvas.cartesian()

axes.bars(np.histogram(posterior_preds[:,-1], bins=50))

0.0 0.3 0.6 0.9 0 10 20 30 40 50 
 
 
 Save as .csv

# Model based on points scored

Note: this didn't turn out to work so well. Probably because there is too much variability that is unmodeled to be able to infer offensive and defensive qualities of teams with only limited data.

In [52]:
stan_scores = """
data {
    int<lower=0> nteams; // number of teams
    int<lower=0> N; // number of observations
    
    vector[N] ptsf; // Points for
    vector[N] ptsa; // Points against
    
    int tid[N]; // Team
    int oid[N]; // Opponent
    real at_home[N]; // Indicator for at home games
    
    int Nnew; // new observations
    int tid_n[Nnew];
    int oid_n[Nnew];
    real at_home_n[Nnew];
    
    real<lower=0> df;
}
transformed data {}
parameters {
    real intercept;
    real home_offense;
    real home_defense;
    
    vector[nteams] offense;
    vector[nteams] defense;
    
    real mu_home;
    real<lower=1> sigma_home;
    
    real<lower=1> sigma_offense;
    real<lower=1> sigma_defense;
    
    real<lower=1> sigma_y;
}
transformed parameters {
    vector[N] xoff;
    vector[N] xdef;
    
    for(i in 1:N) {
        xoff[i] <- intercept + home_offense * at_home[i] + offense[tid[i]] - defense[oid[i]];
        xdef[i] <- intercept - home_defense * at_home[i] + offense[oid[i]] - defense[tid[i]];
    }
}
model {
    intercept ~ normal(22, 10);
    
    home_offense ~ normal(mu_home, sigma_home);
    home_defense ~ normal(mu_home, sigma_home);
    
    offense ~ normal(0, sigma_offense);
    defense ~ normal(0, sigma_defense);

    for(i in 1:N) {
        ptsf[i] ~ student_t(df, xoff, sigma_y);
        ptsa[i] ~ student_t(df, xdef, sigma_y);
    }
}
generated quantities {
    vector[Nnew] xoff_n;
    vector[Nnew] xdef_n;
    vector[Nnew] ptsf_n;
    vector[Nnew] ptsa_n;
    
    for(i in 1:Nnew) {
        xoff_n[i] <- intercept + home_offense * at_home_n[i] + offense[tid_n[i]] - defense[oid_n[i]];
        xdef_n[i] <- intercept - home_defense * at_home_n[i] + offense[oid_n[i]] - defense[tid_n[i]];
        
        ptsf_n[i] <- student_t_rng(df, xoff_n[i], sigma_y);
        ptsa_n[i] <- student_t_rng(df, xdef_n[i], sigma_y);
    }
}
"""

data_scores = {
    "nteams": 32,
    "N": len(traingames),
    "ptsf": traingames.ptsf,
    "ptsa": traingames.ptsa,
    "tid": traingames.tid+1,
    "oid": traingames.oid+1,
    "at_home": traingames.at_home,
    "Nnew": len(testgames),
    "tid_n": testgames.tid+1,
    "oid_n": testgames.oid+1,
    "at_home_n": testgames.at_home,
    "df": 15
}

In [53]:
fit_scores = ps.stan(model_code=stan_scores, data=data_scores, iter=500, chains=2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_472ba2a74d684bbab04bb8c7f3add962 NOW.
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)


In [57]:
posterior_preds_ptsf = fit_scores.extract()["ptsf_n"]
posterior_preds_ptsa = fit_scores.extract()["ptsa_n"]

In [61]:
canvas = tp.Canvas(500, 300)
axes = canvas.cartesian()

axes.scatterplot(posterior_preds_ptsf[:,1], posterior_preds_ptsa[:,1])

0 25 50 75 -30 0 30 60 
 
 
 Save as .csv